In [10]:
#EX1
import numpy as np
import cv2 as cv
import sys
import math


def draw_corners(image, corners_map):
    """Draw a point for each possible corner."""
    for corner in corners_map:
        cv.circle(image,(corner[1], corner[0]),5,(255,0,0),1)

	
	

def moravec(image, threshold = 100):
    """Moravec's corner detection for each pixel of the image."""

    corners = []
    xy_shifts = [(1, 0), (1, 1), (0, 1), (-1, 1)]
    width, height = image.shape[:2]
    for y in range(1, height-1):
        for x in range(1, width-1):
            # Look for local maxima in min(E) above threshold:
            E = 100000
            for shift in xy_shifts:
                diff = int(image[x + shift[0], y + shift[1]])
                diff = int(diff - image[x, y])
                diff = int(diff * diff)

                if (diff < E):
                    E = diff
            if E > threshold:
                corners.append((x, y))

    print("Number of detected features: "+str(len(corners)))
    return corners



threshold = 1000

img = cv.imread('PST_1920x1088_cam4.png')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
corners = moravec(gray, threshold)
draw_corners(img, corners)
cv.imwrite('moravec_'  + str(threshold)+'.png',img)

Number of detected features: 0


True

In [11]:
#EX2
import numpy as np
import cv2 as cv
import sys
import math


def draw_corners(image, corners_map):
    """Draw a point for each possible corner."""
    for corner in corners_map:
        cv.circle(image,(corner[1], corner[0]),5,(255,0,0),1)



def harris(image, threshold = 100000000, sigma = 1.5, k = 0.04):
    """Harris' corner detection for each pixel of the image."""

    corners = []
    width, height = image.shape[:2]
	
    # Calculate gradients:
    X2 = [[0] * width for y in range(height)]
    Y2 = [[0] * width for y in range(height)]
    XY = [[0] * width for y in range(height)]
	
    for y in range(1, height-1):
        for x in range(1, width-1):
            X = int(image[x + 1, y]) - int(image[x - 1, y])
            Y = int(image[x, y + 1]) - int(image[x, y - 1])

            X2[y][x] = int(X * X)
            Y2[y][x] = int(Y * Y)
            XY[y][x] = int(X * Y)

    # Gaussian 3x3:
    G = [[0,0,0], [0,0,0], [0,0,0]]
    for y in range(3):
        for x in range(3):
            u, v = x-1, y-1
            G[y][x] = (math.exp(-(u*u + v*v)/(2*sigma*sigma)))

    # Convolve with Gaussian 3x3:
    A = [[0] * width for y in range(height)]
    B = [[0] * width for y in range(height)]
    C = [[0] * width for y in range(height)]
	
    for y in range(1, height-1):
        for x in range(1, width-1):
            for i in range(3):
                for j in range(3):
                    u, v = j-1, i-1
                    A[y][x] = A[y][x] + X2[y + v][x + u] * G[i][j]
                    B[y][x] = B[y][x] + Y2[y + v][x + u] * G[i][j]
                    C[y][x] = C[y][x] + XY[y + v][x + u] * G[i][j]
    del X2, Y2, XY

    # Harris Response Function:
    R = [[0] * width for y in range(height)]
    for y in range(height):
        for x in range(width):
            a, b, c = A[y][x], B[y][x], C[y][x]
            Tr = a + b
            Det = a * b - c * c
            R[y][x] = Det - k * Tr * Tr
    del A, B, C

    # Suppress Non-Maximum Points:
    for y in range(1, height-1):
        for x in range(1, width-1):
            maximum = True
            for dy in (-1, 0, 1):
                for dx in (-1, 0, 1):
                    if R[y][x] < R[y + dy][x + dx]:
                        maximum = False 
            if maximum and R[y][x] > threshold:
                corners.append((x, y))
            
			
    print("Number of detected features: "+str(len(corners)))
    return corners





threshold = 1000



img = cv.imread('PST_1920x1088_cam4.png')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
corners = harris(gray, threshold)
draw_corners(img, corners)
cv.imwrite('harris_'  + str(threshold)+'.png',img)

Number of detected features: 77851


True

In [32]:
#TASK 1 
#Task 1: Prepare the code for a programme that will determine the dependence of the number
#of feature points determined by Moravec and Harris detectors as a function of the change in
#image quality. Filter the input images with a low pass filter. Put the value of PSNR on the x axis
#and the value of the number of landmarks on the y axis.

#EX2
import numpy as np
import cv2 as cv
import sys
import math
import matplotlib.pyplot as plt

abc=0

threshold = 1000
harrisList = []
moraviec = []
psnr = []

for x in range(1,12,2):
    img = cv.imread('PST_1920x1088_cam4.png')
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    dst = cv.GaussianBlur(gray,(x,x),0,0,0)
    corners = moravec(dst, threshold)
    draw_corners(img, corners)
    cv.imwrite('moravec_'  + str(threshold)+'.png',img)
    moraviec.append(len(corners))

    thresholdHarris = 100000000
    img2 = cv.imread('PST_1920x1088_cam4.png')
    gray2 = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
    dst2 = cv.GaussianBlur(gray2,(x,x),0,0,0)
    corners2 = harris(dst2, thresholdHarris)
    draw_corners(img2, corners2)
    cv.imwrite('harris_'  + str(thresholdHarris)+'.png',img2)
    harrisList.append(len(corners2))

    print(cv.PSNR(dst,dst2),"PSNR")
    psnr.append(cv.PSNR(dst,dst2))

plt.plot(psnr,moraviec)
plt.plot(psnr,harrisList)
plt.ylabel('PSNR vs Landmarks number')
plt.show()

Number of detected features: 598
Number of detected features: 611
361.20199909921956 PSNR
Number of detected features: 33
Number of detected features: 159
361.20199909921956 PSNR
Number of detected features: 3
Number of detected features: 34
361.20199909921956 PSNR
Number of detected features: 0
Number of detected features: 3
361.20199909921956 PSNR
Number of detected features: 0
Number of detected features: 0
361.20199909921956 PSNR
Number of detected features: 0
Number of detected features: 0
361.20199909921956 PSNR


KeyboardInterrupt: 

In [13]:
#EX3
import cv2
import numpy as np

filename = 'chessboard.jpg'
img = cv2.imread(filename)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]

rows,cols = img.shape[1],img.shape[0]
# you can use other parameters
#M = cv2.getRotationMatrix2D((cols/2,rows/2), 45, 1);
#img = cv2.warpAffine(img,M,(cols,rows))

cv2.imshow('dst',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
#EX4
from __future__ import print_function
import cv2 as cv
import numpy as np
import argparse
source_window = 'Obraz zrodlowy'
corners_window = 'Wyznaczone narozniki'
max_thresh = 255

def demo(val):
    thresh = val

    # Parametry detektora
    blockSize = 2
    apertureSize = 3
    k = 0.04
    
    # Wyznaczanie naroznikow
    dst = cv.cornerHarris(src_gray, blockSize, apertureSize, k)
    
    
    # Normalizacja
    dst_norm = np.empty(dst.shape, dtype=np.float32)
    cv.normalize(dst, dst_norm, alpha=0, beta=255, norm_type=cv.NORM_MINMAX)
    dst_norm_scaled = cv.convertScaleAbs(dst_norm)
    
    
    # Wykreślanie okręgów wokół narożników 
    for i in range(dst_norm.shape[0]):
        for j in range(dst_norm.shape[1]):
            if int(dst_norm[i,j]) > thresh:
                cv.circle(dst_norm_scaled, (j,i), 5, (0), 2)
                
    # Wyświetlanie wyników
    cv.namedWindow(corners_window)
    cv.imshow(corners_window, dst_norm_scaled)
    
    
# Ładowanie danych wejściowych
src = cv.imread('building.jpg')
if src is None:
    print('Nie można wczytać obrazka:', args.input)
    exit(0)
src_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)

# Tworzenie okna i suwaka dla parametru threshold
cv.namedWindow(source_window)
thresh = 100 # początkowa wartość progu
cv.createTrackbar('Prog: ', source_window, thresh, max_thresh, demo)
cv.imshow(source_window, src)
demo(thresh)
cv.waitKey()

# thresholding parameter directly influence on number of marked cornetrs. Lower threshold equal more marked corners

[ WARN:0] global /tmp/pip-req-build-agffqapq/opencv/modules/highgui/src/window.cpp (703) createTrackbar UI/Trackbar(Prog: @Obraz zrodlowy): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.


113

In [15]:
#EX5
import cv2
import numpy as np


img = cv2.imread('building.jpg',cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('chessboard.jpg',cv2.IMREAD_GRAYSCALE)



corners = cv2.goodFeaturesToTrack(img, 200, 0.01, 2)
corners = np.int0(corners)

corners2 = cv2.goodFeaturesToTrack(img2, 200, 0.01, 2)
corners2 = np.int0(corners2)
        
for i in corners:
    x,y = i.ravel()
    cv2.circle(img,(x,y),3,1,-1)
    
      
for z in corners2:
    x,y = z.ravel()
    cv2.circle(img2,(x,y),3,1,-1)
    

cv2.imshow('dobre narozniki',img)

cv2.imshow('dobre narozniki2',img2)

cv2.waitKey(0)
cv2.destroyAllWindows()

#Note the effect of the thresholding parameter on the number of corners determined.
#How does the point detection performance differ from the metric used in the Harris
#detector for the same images? In addition to the building.jpg, image, use another
#example image.

# thresholding parameter directly influence on number of marked cornetrs. Lower threshold equal more marked corners. We have 3 parameters
#that have directly influence on this number - maxCorners, qualityLevel, minDistance. Max corners can block numbers of detected cornes
#minDistance limit number of detected corners near each other, qualityLevel Parameter characterizing the minimal accepted quality 
# of image corners. The parameter value is multiplied by the best corner quality measure, which is the minimal eigenvalue 
# (see #cornerMinEigenVal ) or the Harris function response (see #cornerHarris ). The corners with the quality measure less 
# than the product are rejected. For example, if the best corner has the quality measure = 1500, and the qualityLevel=0.01 , 
# then all the corners with the quality measure less than 15 are rejected.
# Harris detector in result gives more points than Moraviec in the same thershold.



In [17]:
#EX5 rotated

#Based on task 1, check how the number of points detected by the Shi-Tomasi detector
#changes as a function of changing the image angle. Show and comment on the
#relationship between number of points and rotation.
import cv2
import numpy as np

img = cv2.imread('chessboard.jpg',cv2.IMREAD_GRAYSCALE)


rows,cols = img.shape[1],img.shape[0]
# you can use other parameters
#M = cv2.getRotationMatrix2D((cols/2,rows/2), 45, 1);
#img = cv2.warpAffine(img,M,(cols,rows))

corners = cv2.goodFeaturesToTrack(img, 200, 0.01, 1)
corners = np.int0(corners)
        
for i in corners:
    x,y = i.ravel()
    cv2.circle(img,(x,y),3,5,-1)

    
print(len(corners))
cv2.imshow('dobre narozniki',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

#Rotated images gives in results less points than straight image

141
